## Foundations of deep learning - HW1


In [43]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm, confusion_matrix
from sklearn.metrics import accuracy_score
from numpy.random import default_rng


***Part 1 - Setup and baseline***

We will begin with downloading the datasets, and getting baseline results using SVM classification.First step is downloading the CIFAR dataset. We will only use a subsample, but we will download the whole set and later sample out the actual train and test sets.

In [44]:
fullTrain = torchvision.datasets.CIFAR10('datasets/CIFAR', train=True, transform=np.array, target_transform=None, download=True)
fullTest  = torchvision.datasets.CIFAR10('datasets/CIFAR', train=False, transform=np.array, target_transform=None, download=True)

Files already downloaded and verified
Files already downloaded and verified


We will now create random masks to represent the chosen subset we will use:

In [45]:
rng = default_rng(42) # keeping dataset consistent
trainMask = rng.choice(len(fullTrain), len(fullTrain)//10, replace=False)
testMask =  rng.choice(len(fullTest), len(fullTest)//10, replace=False)

Narrowing the datasets that we will use:

In [46]:
trainData    = fullTrain.data[trainMask]
trainLabels  = np.array(fullTrain.targets)[trainMask]
testData    = fullTest.data[testMask]
testLabels  = np.array(fullTest.targets)[testMask]
print('Dataset shapes: Train - {}, Test - {}'.format(trainData.shape,testData.shape))

Dataset shapes: Train - (5000, 32, 32, 3), Test - (1000, 32, 32, 3)


Flattening out the tensors:

In [47]:
trainData = trainData.reshape(trainData.shape[0],-1)
testData  = testData.reshape(testData.shape[0],-1)
print('Dataset shapes: Train - {}, Test - {}'.format(trainData.shape,testData.shape))

Dataset shapes: Train - (5000, 3072), Test - (1000, 3072)


Now lets get down to buisness, train and evaluate the SVM baseline model.
To get a reasonable result we will use a grid search to find the best parameters for the regularization constant and the Gamma paramater for the RBF kernel.

starting with the vanila linear SVM:

In [49]:
model = svm.SVC()
C = np.logspace(-5, 5, num=11)
params = {'kernel': ['linear'], 'C': C}
linearSelection = GridSearchCV(model, params, n_jobs=-1, verbose=1)
linearSelection.fit(trainData,trainLabels)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  35 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed: 18.7min


KeyboardInterrupt: 

In [ ]:
print('paramaters chosen for the svm model: {}'.format(linearSelection.best_params_))
prediction = linearSelection.predict(testData, testLables)
score = accuracy_score(testLables, prediction)
print('Preformance on test set:{}'.format(score))
print('confusion matrix:')
print(confusion_matrix(testLables, prediction))

RBF kernel:

In [37]:
model = svm.SVC()
gamma = np.logspace(-10, 10, num=21)
C = np.logspace(-5, 5, num=11)
params = {'kernel': ['rbf'], 'C': C, 'gamma': gamma}
rbfSelection = GridSearchCV(model, params, n_jobs=-1, verbose=1)
rbfSelection.fit(trainData,trainLabels)

KeyboardInterrupt: 

In [ ]:
print('paramaters chosen for the svm model: {}'.format(rbfSelection.best_params_))
prediction = rbfSelection.predict(testData, testLables)
score = accuracy_score(testLables, prediction)
print('Preformance on test set:{}'.format(score))
print('confusion matrix:')
print(confusion_matrix(testLables, prediction))